In [3]:
from instaloader import instaloader, Profile
from datetime import datetime
from itertools import dropwhile, takewhile
L=instaloader.Instaloader(
    download_videos=False,
    download_video_thumbnails = False
)
Liste = [Namen der Accounts]
i=0
while i < len(Liste):
    try:
        PROFILE = (Liste[i])
        profile = Profile.from_username (L.context, PROFILE)
        posts= profile.get_posts()
        SINCE = datetime (2024 , 7, 1)
        UNTIL= datetime (2022, 1, 1)
        for post in takewhile(lambda p: p.date > UNTIL, dropwhile(lambda p: p.date > SINCE, posts)):
            L.download_post(post, PROFILE)
        i +=1 
    except:
        PROFILE= (Liste[i])
        formatted_string = f"Hello,{PROFILE} did not work." 
        print(formatted_string) 
        i +=1           

[BREAKING: AfD-Mitglieder lief…] json 


In [34]:
#Pytesseract alle Dateien und erste Bereinigung 
import glob
import os
import pandas as pd
import nltk
import string
import spacy
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pytesseract
from PIL import Image
import spacy
import cv2
import re
from string import digits
import csv

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR/tesseract.exe'
Liste = [Name der Accounts]
i = 0
while i < len(Liste):
    try:
        start_path = r"PathallerDateien"
        middle_path = os.path.join(start_path, Liste[i])
        final_path = os.path.join(middle_path, "*.jpg")
        dir_list= glob.glob(final_path)
        accountcsv = [Liste[i], ".csv"]
        accountcsvfile = "".join(accountcsv)
        accountfile= os.path.join(middle_path, accountcsvfile )
        k=0
        while k < len(dir_list):
            img= Image.open(dir_list[k])
            img = cv2.imread(dir_list[k])
            HSV_img = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
            h,s,v = cv2.split(HSV_img)
            thresh = cv2.threshold(v, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
            ocr_result1 = pytesseract.image_to_string(thresh, lang="deu", config='psm 7').replace("\n", " ")
            originalImage = cv2.imread(dir_list[k])
            grayImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2GRAY)
            (thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 230, 255, cv2.THRESH_BINARY)
            ocr_result2 = pytesseract.image_to_string(blackAndWhiteImage, lang="deu").replace("\n", " ")
            ocr_result_lemma= []
            if len(ocr_result1) > len(ocr_result2):
                ocr_result_final= ocr_result1
            elif len(ocr_result1) < len(ocr_result2):
                ocr_result_final = ocr_result2
            else :
                ocr_result_final = ocr_result1
            nlp= spacy.load('de_core_news_md')
            doc= nlp(ocr_result_final)
            result = ' '.join([x.lemma_ for x in doc if not x.is_stop]) 
            ocr_result_lemma.append(result)
            remove_digits = str.maketrans('', '', digits)
            punctuation = " ".join(ocr_result_lemma).translate(str.maketrans("","",string.punctuation)).replace("\u2060"," ").lower().translate(remove_digits)
            tokenizer= TweetTokenizer()
            tokens=tokenizer.tokenize(punctuation)
            tokens = [t for t in tokens if len(t) > 3]
            with open (accountfile, "a", encoding="utf-8", newline="") as f:
                write= csv.writer(f)
                write.writerow(tokens)
            k +=1
        i +=1
    except:
        with open ("DateeipfadListeProbleme", "a", encoding="utf-8", newline="") as f:
            write=csv.writer(f)
            write.writerow(Liste[i])
            i +=1
    

In [ ]:
#Bereinigung weiterer Probleme
import csv
import os
from collections import defaultdict
Liste = ["NamenderAccounts"]
i=0
while i < len(Liste):
# Create CSV reader and writer objects
    start_path = r"Pfadallerdateien"
    naming_object = "Bilder_" + Liste[i] + ".csv"
    middle_path = os.path.join(start_path, Liste[i])
    output_path = os.path.join(middle_path, "BilderSauber.csv")
    input_path = os.path.join(middle_path, naming_object)
    with open(input_path, 'r', encoding="utf-8", newline="") as input_file, \
         open(output_path, 'a', encoding="utf-8", newline="") as output_file:
        reader= csv.reader(input_file)
        writer= csv.writer(output_file)
        for row in reader:
            if not row:
                continue  # will loop without writing any output
            row = [field.replace('ÃŸ', 'ß').replace("Ã¤", "ä").replace("Ã¶","ö").replace("Ã¼", "ü").replace("english", "").replace("comment","").replace("below","") for i, field in enumerate(row)]
            writer.writerow(row)
    i +=1